In [165]:
import time

In [117]:
from sklearn.neural_network import MLPClassifier
from sklearn import datasets, preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

In [10]:
iris = datasets.load_iris()

In [104]:
X = preprocessing.normalize(iris.data[:, :2],axis=1)  # we only take the first two features.
y = iris.target

In [132]:
def get_all_weights(clf):
    all_weights = [] 
    for layer in clf.coefs_:
        layers_weights = layer.reshape(1,-1)
        all_weights = np.append(all_weights, layers_weights)
    return all_weights

In [105]:
X

array([[ 0.82451335,  0.5658425 ],
       [ 0.8528513 ,  0.52215386],
       [ 0.82659925,  0.56279098],
       [ 0.82926643,  0.55885346],
       [ 0.81153434,  0.58430473],
       [ 0.81067923,  0.58549055],
       [ 0.80417614,  0.59439106],
       [ 0.8269265 ,  0.56231002],
       [ 0.8349582 ,  0.55031336],
       [ 0.84507884,  0.53464171],
       [ 0.82493237,  0.56523144],
       [ 0.81602448,  0.57801734],
       [ 0.8479983 ,  0.52999894],
       [ 0.82012695,  0.5721816 ],
       [ 0.82321279,  0.56773296],
       [ 0.79159032,  0.61105218],
       [ 0.81067923,  0.58549055],
       [ 0.82451335,  0.5658425 ],
       [ 0.83205029,  0.5547002 ],
       [ 0.80188283,  0.59748132],
       [ 0.84623284,  0.53281327],
       [ 0.80942185,  0.58722762],
       [ 0.787505  ,  0.61630826],
       [ 0.83957016,  0.54325128],
       [ 0.81602448,  0.57801734],
       [ 0.85749293,  0.51449576],
       [ 0.8269265 ,  0.56231002],
       [ 0.82958775,  0.55837637],
       [ 0.83696961,

In [142]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [271]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, max_iter=1, warm_start=True,
                    hidden_layer_sizes=(5), random_state=1)

In [272]:
for i in range(100):
    time.sleep(0.1)
    clf.fit(X_train, y_train)
    print(clf.score(X_test, y_test))
    set_leds_to_weights(get_all_weights(clf))

0.3
0.32
0.32
0.3
0.32
0.32
0.3
0.32
0.32
0.34
0.64
0.34
0.42
0.72
0.66
0.72
0.68
0.72
0.72
0.72
0.72
0.72
0.72
0.72
0.74
0.74
0.72
0.74
0.74
0.74
0.72
0.74
0.72
0.74
0.74
0.74
0.7
0.74
0.72
0.74
0.72
0.74
0.72
0.74
0.72
0.74
0.72
0.74
0.7
0.74
0.7
0.74
0.72
0.74
0.7
0.74
0.72
0.74
0.72
0.74
0.72
0.74
0.72
0.74
0.72
0.74
0.72
0.76
0.72
0.68
0.74
0.72
0.68
0.74
0.72
0.7
0.74
0.7
0.74
0.72
0.72
0.7
0.7
0.7
0.7
0.7
0.72
0.7
0.7
0.7
0.72
0.7
0.68
0.72
0.7
0.76
0.72
0.7
0.68
0.7


In [172]:
from led_api import led_api

In [176]:
led_data_to_send = {"leds":[]}
for i in range(30):
    to_add = { "red": 0, "green": 0, "blue": 0, "led_num": i }
    led_data_to_send['leds'].append(to_add)

In [178]:
led_api.send_led_json(led_data_to_send)

In [263]:
def set_leds_to_weights(weights):
    led_data_to_send = {"leds":[]}
    for i,w in np.ndenumerate(weights):
        norm_weight = max(-1, min(w, 1))+1
        (r,g,b) = val_to_rgb(norm_weight)
        to_add = { "red": r, "green": g, "blue": b, "led_num": i[0] }
        led_data_to_send['leds'].append(to_add)
    led_api.send_led_json(led_data_to_send)

In [245]:
set_leds_to_weights(get_all_weights(clf))

In [181]:
w = get_all_weights(clf)

In [183]:
for wd in w:
    print(wd)

-0.153644337444
0.0782405724618
-0.92560196717
-1.55066383451
-0.654076504463
-0.754837065946
-0.782507690232
-0.285964003876
2.05278521227
0.0718741318866
0.295256362584
-0.143231300972
0.1016530672
-0.557574347172
-0.450423728524
0.383135583572
0.811047274873
-0.323156585352
0.333110255941
2.16087710204
-0.254659397772
-1.28954251939
-0.7983750564
-0.57186656421
0.654957534043


In [186]:
for i,v in np.ndenumerate(w):
    print (v,i[0])

-0.153644337444 0
0.0782405724618 1
-0.92560196717 2
-1.55066383451 3
-0.654076504463 4
-0.754837065946 5
-0.782507690232 6
-0.285964003876 7
2.05278521227 8
0.0718741318866 9
0.295256362584 10
-0.143231300972 11
0.1016530672 12
-0.557574347172 13
-0.450423728524 14
0.383135583572 15
0.811047274873 16
-0.323156585352 17
0.333110255941 18
2.16087710204 19
-0.254659397772 20
-1.28954251939 21
-0.7983750564 22
-0.57186656421 23
0.654957534043 24


In [215]:
import colorsys

In [234]:
colorsys.hsv_to_rgb(0.67,1,1)

(0.020000000000000462, 0.0, 1)

In [239]:
def val_to_rgb(val):
    i = val*0.67
    c = colorsys.hsv_to_rgb(i,1,1)
    return [int(color*255) for color in c]

In [243]:
val_to_rgb(0.5)

[0, 255, 2]